# Stacks PoX Flywheel: Fees → Miner Rewards → BTC Bids → PoX Yields

This notebook reconstructs the Stacks Proof-of-Transfer (PoX) flywheel by joining fees, miner rewards, BTC bids, and stacker yields at the burn-block (tenure) level. It relies on the Signal21 public API for market and fee data and the Hiro Stacks API for burn chain metadata.

**Objectives**
- Pull full-history STX/BTC prices, transaction fees, PoX rewards, and anchor metadata.
- Construct a tenure-level panel with derived reward value and \(ho = rac{	ext{BTC commit}}{	ext{reward value}}\).
- Quantify fee uplift scenarios (+10/25/50/100/200%) to estimate incremental BTC commits and PoX APY shifts.

**References**
- [Signal21 API docs](https://app.signal21.io/docs/api.html)
- [Signal21 API access](https://signal21.github.io/docs/extras/api-access.html)
- [Hiro Stacks API reference](https://www.hiro.so/stacks-api)
- [Stacks fee mechanics](https://docs.stacks.co/concepts/network-fundamentals/network)
- [Stacks mempool fee endpoint](https://www.quicknode.com/docs/stacks/v2/extended-v2-mempool-fees)

In [ ]:
import os
from pathlib import Path
import subprocess
import sys

if "google.colab" in sys.modules:
    repo_path = Path('/content/stx-labs')
    if repo_path.exists():
        subprocess.run(['git', '-C', str(repo_path), 'fetch', '--all'], check=True)
        subprocess.run(['git', '-C', str(repo_path), 'checkout', 'main'], check=True)
        subprocess.run(['git', '-C', str(repo_path), 'pull', '--ff-only'], check=True)
    else:
        subprocess.run(['git', 'clone', 'https://github.com/seconds-0/stx-labs.git', str(repo_path)], check=True)
    os.chdir(repo_path)
    subprocess.run(['pip', 'install', '--quiet', '-r', 'requirements.txt'], check=True)
    print('Colab environment ready: repo synced and dependencies installed.')
else:
    print('Running outside Colab; ensure you execute from the repo root.')


## 1. Configuration & Environment Checks

Edit the cell below to configure date windows, retry policy, and manual overrides. Set `HIRO_API_KEY` in your environment (or in the notebook UI) before running the data acquisition cells.

In [ ]:
from __future__ import annotations

import os
from datetime import datetime, timedelta
from pathlib import Path

import pandas as pd

from src import config as cfg

# src.config automatically loads .env (HIRO_API_KEY, etc.)

# -------- User Parameters -------- #
WINDOW_DAYS = (30, 90, 180)
USE_FULL_HISTORY = True  # pull as far back as available when True
START_DATE = datetime.utcnow() - timedelta(days=5 * 365)
END_DATE = datetime.utcnow()
FORCE_REFRESH = False  # set True to bypass local cache

# Scenario assumptions
COINBASE_STX = 1_000.0
FEE_PER_TX_STX = 0.08
RHO_RANGE = (0.3, 0.5, 0.7)
UPLIFT_POINTS = (0.10, 0.25, 0.50, 1.00, 2.00)
REWARD_BLOCKS_PER_CYCLE = 2100

RAW_PATH = cfg.RAW_DATA_DIR
OUT_PATH = cfg.OUT_DIR
RAW_PATH.mkdir(parents=True, exist_ok=True)
OUT_PATH.mkdir(parents=True, exist_ok=True)

HIRO_API_KEY = os.getenv(cfg.HIRO_API_KEY_ENV)
if not HIRO_API_KEY:
    print("⚠️ Set HIRO_API_KEY before running Hiro API calls.")

## 2. Imports & Helper Setup

The helper modules live in `src/` and encapsulate Signal21/Hiro API access, caching, and scenario math.

In [ ]:
import json
from collections import defaultdict

import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from src import hiro, panel_builder, prices, scenarios
from src.fees import fetch_fee_per_tx_summary, fetch_fees_by_tenure
from src.signal21 import probe_schema

pd.options.display.float_format = "{:.6f}".format

## 3. Schema Discovery (Signal21)

Run these exploratory queries intermittently to confirm dataset layouts. Results are cached locally.

In [ ]:
# Uncomment to inspect schema when needed
# tx_sample = probe_schema("core.txs")
# block_sample = probe_schema("core.blocks")
# display(tx_sample.head())
# display(block_sample.head())

## 4. Data Acquisition

This section ingests prices, fees, PoX rewards, and anchor metadata. Each request uses robust retry logic and caches raw payloads under `data/raw/`.

In [ ]:
price_start = START_DATE if USE_FULL_HISTORY else datetime.utcnow() - timedelta(days=max(WINDOW_DAYS))
price_end = END_DATE
prices_df = prices.load_price_panel(price_start, price_end, force_refresh=FORCE_REFRESH)
print(f"Loaded {len(prices_df)} hourly price records spanning {prices_df['ts'].min()} to {prices_df['ts'].max()}")
prices_df.head()

In [ ]:
fees_df = fetch_fees_by_tenure(force_refresh=FORCE_REFRESH)
print(f"Fetched fee aggregates for {fees_df.shape[0]} burn blocks")
fees_df.head()

In [ ]:
if fees_df.empty:
    rewards_df = pd.DataFrame()
else:
    min_height = int(fees_df['burn_block_height'].min())
    max_height = int(fees_df['burn_block_height'].max())
    rewards_df = hiro.aggregate_rewards_by_burn_block(start_height=min_height, end_height=max_height, force_refresh=FORCE_REFRESH)
    print(f"Aggregated PoX rewards for {rewards_df.shape[0]} burn blocks")
rewards_df.head()

In [ ]:
if fees_df.empty:
    anchor_df = pd.DataFrame()
else:
    anchor_df = hiro.collect_anchor_metadata(fees_df['burn_block_height'].astype(int), force_refresh=FORCE_REFRESH)
    print(f"Collected anchor metadata for {anchor_df.shape[0]} burn blocks")
anchor_df.head()

In [ ]:
cycles_df = hiro.list_pox_cycles(force_refresh=FORCE_REFRESH)
print(f"Retrieved {cycles_df.shape[0]} PoX cycles")
cycles_df.head()

## 5. Tenure Panel Construction

Join all datasets on `burn_block_height`, align prices to anchor timestamps, and derive reward value and \(ho\).

In [ ]:
if fees_df.empty or anchor_df.empty:
    panel_df = pd.DataFrame()
else:
    panel_cfg = panel_builder.PanelConfig(coinbase_stx=COINBASE_STX)
    panel_df = panel_builder.build_tenure_panel(
        fees=fees_df,
        rewards=rewards_df,
        anchors=anchor_df,
        prices=prices_df,
        config=panel_cfg,
    )
    panel_df = panel_builder.merge_cycle_metadata(panel_df, cycles_df)
    print(f"Panel contains {panel_df.shape[0]} tenures")
panel_df.head()

## 6. Validation Checks

Ensure we have consistent tenure coverage, expected coinbase value, and reasonable \(ho\) ranges.

In [ ]:
if not panel_df.empty:
    missing_fees = panel_df['fees_stx_sum'].isna().sum()
    coinbase_anomalies = panel_df['coinbase_flag'].sum()
    rho_div_zero = panel_df['rho_flag_div0'].sum()
    print("Missing fee entries:", missing_fees)
    print("Coinbase anomalies:", coinbase_anomalies)
    print("Zero reward value entries:", rho_div_zero)

    expected_burns = panel_df['burn_block_height'].iloc[-1] - panel_df['burn_block_height'].iloc[0] + 1
    missing_burns = expected_burns - len(panel_df['burn_block_height'].unique())
    print("Missing burn heights:", missing_burns)

    sample = panel_df.sample(min(20, len(panel_df)))
    sample[['burn_block_height', 'fees_stx_sum', 'reward_amount_sats_sum', 'rho']]

## 7. Fee Analytics Per Window

Compute empirical fee-per-transaction statistics across rolling windows to benchmark against the 0.08 STX/tx baseline.

In [ ]:
fee_stats = {}
for window in WINDOW_DAYS:
    stats_df = fetch_fee_per_tx_summary(window, force_refresh=FORCE_REFRESH)
    fee_stats[window] = stats_df
    print(f"Window {window}d -> observations: {stats_df.shape[0]}")
fee_summary_df = pd.concat({f"{w}d": df for w, df in fee_stats.items()}, names=["window", "row"])
fee_summary_df.groupby(level="window").agg({"avg_fee_stx": "mean", "median_fee_stx": "mean"})

## 8. Scenario Engine

Estimate the incremental transactions, BTC commits, and PoX APY shifts for fee uplifts. `stacked_supply_stx` defaults to a rolling estimate when available, otherwise falls back to 1.35B STX.

In [ ]:
if panel_df.empty:
    scenario_df = pd.DataFrame()
else:
    recent_panel = panel_df.tail(max(3_000, len(panel_df)))
    mean_fee_stx = recent_panel['fees_stx_sum'].mean()
    mean_stx_btc = recent_panel['stx_btc'].mean()
    stacked_supply_estimate = (
        recent_panel['reward_stx_total'].rolling(REWARD_BLOCKS_PER_CYCLE).sum().dropna().iloc[-1]
        if len(recent_panel) >= REWARD_BLOCKS_PER_CYCLE
        else 1_350_000_000.0
    )
    scenario_cfg = scenarios.ScenarioConfig(
        fee_per_tx_stx=FEE_PER_TX_STX,
        rho_candidates=RHO_RANGE,
        coinbase_stx=COINBASE_STX,
        reward_cycles_blocks=REWARD_BLOCKS_PER_CYCLE,
        stacked_supply_stx=stacked_supply_estimate,
    )
    scenario_df = scenarios.build_scenarios(
        uplift_rates=UPLIFT_POINTS,
        mean_fee_stx=mean_fee_stx,
        mean_stx_btc=mean_stx_btc,
        config=scenario_cfg,
    )
    scenario_df

## 9. Visualizations

Produce canonical charts: time series of fees and BTC commits, \(ho\) distribution, and scatter of reward value vs. BTC commits.

In [ ]:
if not panel_df.empty:
    time_fig = px.line(
        panel_df.tail(5_000),
        x="burn_block_time_iso",
        y=["fees_stx_sum", "reward_amount_sats_sum"],
        labels={"value": "Amount", "burn_block_time_iso": "Burn Block Time"},
        title="Tenure Fees vs. BTC Commit (Rolling)",
    )
    time_fig.show()

    rho_fig = px.histogram(panel_df, x="rho", nbins=50, title="Distribution of Rho")
    rho_fig.show()

    scatter_fig = px.scatter(
        panel_df,
        x="reward_value_sats",
        y="reward_amount_sats_sum",
        trendline="ols",
        title="BTC Commit vs. Reward Value",
        labels={"reward_value_sats": "Reward Value (sats)", "reward_amount_sats_sum": "BTC Commit (sats)"},
    )
    scatter_fig.show()

## 10. Artifact Export

Persist key datasets to `./data/` and `./out/` for downstream usage.

In [ ]:
if not panel_df.empty:
    panel_path = OUT_PATH / "tenure_panel.parquet"
    fees_path = OUT_PATH / "fees_by_tenure.parquet"
    rewards_path = OUT_PATH / "pox_rewards.parquet"
    price_path = OUT_PATH / "prices.parquet"
    scenario_path = OUT_PATH / "scenario_table.csv"

    panel_df.to_parquet(panel_path, index=False)
    fees_df.to_parquet(fees_path, index=False)
    rewards_df.to_parquet(rewards_path, index=False)
    prices_df.to_parquet(price_path, index=False)
    scenario_df.to_csv(scenario_path, index=False)

    print("Saved panel ->", panel_path)
    print("Saved fees ->", fees_path)
    print("Saved rewards ->", rewards_path)
    print("Saved prices ->", price_path)
    print("Saved scenario table ->", scenario_path)

## 11. Next Steps

- Extend to cycle-level aggregates (sum commits, rewards, rho by PoX cycle).
- Add pool attribution analytics by incorporating stacker addresses.
- Compare realized fees with Hiro mempool fee estimates for additional validation.
- Integrate notebook with Deepnote (recommended) for easy sharing; sync with this repo for reproducibility.